In [105]:
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
import os
import json

In [106]:
# Метод получения списка слов (в нижнем регистре) из текста
def get_words_from_text(text):
    sentences = sent_tokenize(text)
    words = []
    for sentence in sentences:
        words = words + [word.lower() for word in word_tokenize(sentence)]
    return words

In [107]:
text_files_indices_with_words = {}
total_texts_count = 0
data_dir_path = '../Выкачка/'

# Проходим по файлам выкачек, собираем общее количество файлов
# Также сохраняем словарь индекс_файла: список слов в файле
for filename in os.listdir(data_dir_path):
    if 'выкачка' not in filename:
        continue
    total_texts_count += 1
    full_file_path = data_dir_path + filename
    with open(full_file_path, 'r') as f:
        text_files_indices_with_words[total_texts_count] = get_words_from_text(f.read())

In [108]:
# Метод подсчета tf для термина = word в тексте файла
def calculate_token_tf(text_index, word):
    text_words = text_files_indices_with_words[text_index]
    word_occurence_count = text_words.count(word.lower())
    return word_occurence_count / len(text_words)

In [109]:
# Метод подсчета idf для термина
def calculate_token_idf(word):
    word_occurence_in_texts_count = 0
    for text_file_index in text_files_indices_with_words:
        text_words = text_files_indices_with_words[text_file_index]
        if word in text_words:
            word_occurence_in_texts_count += 1
    return word_occurence_in_texts_count / total_texts_count

In [110]:
# Получаем список всех токенов, собранных при токенизации (см. tokenizer.ipynb)
tokens = []
with open('../tokens.txt', 'r') as f:
    tokens = f.read().split('\n')
    
for text_file_index in text_files_indices_with_words:
    # Получаем список слов для каждого текста
    text_words = text_files_indices_with_words[text_file_index]
    token_tf_idf_file_path = '../tf_idf_freq/tokens/tokens_' + str(text_file_index) + '.txt'
    # Для каждого токена, если он содержится в файле подсчитываем его параметры tf-idf-tf_idf
    # Параметры записываются в соответствующий файл tokens_${file_index}.txt
    with open(token_tf_idf_file_path, 'w') as tokens_tf_idf_file:
        for word in text_words:
            if word not in tokens:
                continue
            tf = calculate_token_tf(text_file_index, word)
            idf = calculate_token_idf(word)
            tf_idf = tf * idf
            tokens_tf_idf_file.write(word + ' ' + str(tf) + ' ' + str(idf) + ' ' + str(tf_idf) + '\n')

In [111]:
inverted_index_dict = {}

# Получаем словарь инвертированного индекса, полученного в inverted_index.ipynb
with open('../inverted_index.txt', 'r') as index_file:
    for line in index_file:
        line_json = json.loads(line)
        inverted_index_dict[line_json['word']] = line_json['inverted_array']

In [112]:
lemmas = {}

# Получаем словарь лемма: список токенов, полученных в результате лемматизации в tokenizer.ipynb
with open('../lemmas.txt', 'r') as f:
    for line in f:
        lemma_with_tokens = line.split(':')
        lemma = lemma_with_tokens[0]
        words = [word.strip() for word in lemma_with_tokens[1].split(' ') if word != '']
        lemmas[lemma] = words

In [113]:
# Метод подсчета tf для леммы
def calculate_lemma_tf(text_index, lemma):
    lemma_words = lemmas[lemma]
    lemma_count = 0
    for word in lemma_words:
        lemma_count += calculate_token_tf(text_index, word)
    return lemma_count / len(text_words)

In [114]:
# Метод подсчета idf для леммы
def calculate_lemma_idf(lemma):
    lemma_files_indices = inverted_index_dict[lemma]
    return len(lemma_files_indices) / total_texts_count

In [115]:
for text_file_index in text_files_indices_with_words:
    lemma_tf_idf_file_path = '../tf_idf_freq/lemmas/lemmas_' + str(text_file_index) + '.txt'
    # Для каждой леммы, если она содержится в файле подсчитываем ее параметры tf-idf-tf_idf
    # Параметры записываются в соответствующий файл lemmas_${file_index}.txt
    with open(lemma_tf_idf_file_path, 'w') as lemmas_tf_idf_file:
        for lemma in lemmas:
            if text_file_index in inverted_index_dict[lemma]:
                tf = calculate_lemma_tf(text_file_index, lemma)
                idf = calculate_token_idf(lemma)
                tf_idf = tf * idf
                lemmas_tf_idf_file.write(lemma + ' ' + str(tf) + ' ' + str(idf) + ' ' + str(tf_idf) + '\n')